In [ ]:
import keepa
import pandas as pd
import time
import inspect
import csv
from datetime import date
from datetime import datetime

This code makes API requests to keepa based on the asins from the raw data to retrieve historical and product data

In [ ]:
accesskey = 'gfe071s7bsqhvf126ac7r1v7ug770q8uu1rtgleh4tprpq3oesuuqvatfo7lii50'
api = keepa.Keepa(accesskey)

In [ ]:
#Historical data retrieved from keepa are in dataframe format, sometimes with multiple observations for each date
#Sicne amazone does not provide information of time in review, a simple average was used to calculated
def transform_df(df):
    t_df = df
    try:
        t_df.index = df.index.strftime('%Y-%m-%d')
    except:
        print('date already formatted')
#-1 means missing. This is removed first
    t_df = t_df[t_df.value != -1]
    t_df = t_df.groupby(t_df.index).mean()
    return t_df

In [ ]:
def retrieve_data(asin, products):
    if len(products[0]['data']) != 0:
        #price cheapest new offering
        price_new_history = transform_df(products[0]['data']['df_NEW']).rename({'value' : 'price_new'}, axis=1)
        #price cheapest used product offering
        price_used_history = transform_df(products[0]['data']['df_USED']).rename({'value' : 'price_used'}, axis=1)
        #Star ratings
        try:
            rating_history = transform_df(products[0]['data']['df_RATING']).rename({'value' : 'rating'}, axis=1)
        except:
            rating_history = pd.DataFrame({'rating' : None}, index=[date.today()])
        #Number of reviews
        try:
            review_history = transform_df(products[0]['data']['df_COUNT_REVIEWS']).rename({'value' : 'reviews'}, axis=1)
        except:
            review_history = pd.DataFrame({'reviews' : None}, index=[date.today()])
        #Merge everything with outer joins
        df = price_new_history.merge(price_used_history, right_index = True, left_index = True, how = "outer").merge(
        rating_history, right_index = True, left_index = True, how = "outer").merge(
        review_history, right_index = True, left_index = True, how = "outer")
    #If there's no historical data
    else:
        df = pd.DataFrame({'price_new':products[0]['stats']['avg'][1], 
                           'price_used' : products[0]['stats']['avg'][2], 
                           'rating' : None, 
                           'reviews' : None}, index=[date.today()])
    df["asin"] = asin
    #Out of stock new products
    df["oos_new"] = products[0]['stats']["outOfStockPercentageInInterval"][1]
    #Out of stock used products
    df["oos_used"] = products[0]['stats']["outOfStockPercentageInInterval"][2]
    #Number of keepa minutes since product was listed
    df["listed_since"] = products[0]['listedSince']
    df.index = pd.to_datetime(df.index)
    #df = df.interpolate(method='time')
    return df

In [ ]:
#import data from csv file
data = pd.read_csv('../../gen/input/amazon_search_hedonic_clean.csv', sep = ";")
asins = data['asin'].unique()

In [ ]:
with open(f"../../gen/input/api_info.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["date", "price_new", "price_used", "rating", "reviews", "asin", "oos_new", "oos_used", "listed_since"])

In [ ]:
data_so_far = pd.read_csv(f"../../gen/input/api_info.csv", sep = ";")
asins_so_far = data_so_far['asin'].unique()

print(len(asins))
print(len(asins_so_far))
asins_left = list(set(asins) - set(asins_so_far))
len(asins_left)


In [ ]:
data_so_far = pd.read_csv(f"../../gen/input/api_info.csv", sep = ";")
asins_so_far = data_so_far['asin'].unique()

print(len(asins))
print(len(asins_so_far))
asins_left = list(set(asins) - set(asins_so_far))
print(len(asins_left))

i = 0
tokens_left = api.tokens_left
print("tokens left: ", tokens_left)
#for asin in asins_left[3:round(((tokens_left*1.1)/2))]:
for asin in asins_left:
    print(asin)
    products = api.query(asin, stats = 3650, history = 1, rating = 1)
    try:
        df = retrieve_data(asin, products)
        #df1 = df.interpolate(method='time')
        #df2 = df1.fillna(method='bfill')
        df.to_csv(f"../../gen/input/api_info.csv", mode = "a", sep = ";", header = False)
    except:
        print("error with asin: ", asin)
        continue
    i += 1
    if i % 10 == 0:
        print(i)
    time.sleep(0.3)